<a href="https://colab.research.google.com/github/bsong75/brendensong.github.io/blob/main/4_a_NLP_McDonald's_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
mcd = pd.read_excel('/content/drive/MyDrive/csv/McD.xlsx')


In [12]:
mcd.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,policies_violated,policies_violated:confidence,city,policies_violated_gold,review
0,679455653,False,finalized,3,2015-02-21 00:36:00,RudeService\rOrderProblem\rFilthy,1.0\r0.6667\r0.6667,Atlanta,NaN,"I'm not a huge mcds lover, but I've been to be..."
1,679455654,False,finalized,3,2015-02-21 00:27:00,RudeService,1,Atlanta,NaN,Terrible customer service. Î¾I came in at 9:30...
2,679455655,False,finalized,3,2015-02-21 00:26:00,SlowService\rOrderProblem,1.0\r1.0,Atlanta,NaN,"First they ""lost"" my order, actually they gave..."
3,679455656,False,finalized,3,2015-02-21 00:27:00,na,0.6667,Atlanta,NaN,I see I'm not the only one giving 1 star. Only...
4,679455657,False,finalized,3,2015-02-21 00:27:00,RudeService,1,Atlanta,NaN,"Well, it's McDonald's, so you know what the fo..."


In [13]:
mcd.loc[0,'review']

"I'm not a huge mcds lover, but I've been to better ones. This is by far the worst one I've ever been too! It's filthy inside and if you get drive through they completely screw up your order every time! The staff is terribly unfriendly and nobody seems to care."

In [14]:
# examine the shape before removing any rows
mcd.shape

# count the number of null values in each column
mcd.isna().sum()


_unit_id                           0
_golden                            0
_unit_state                        0
_trusted_judgments                 0
_last_judgment_at                  0
policies_violated                 54
policies_violated:confidence      54
city                              87
policies_violated_gold          1525
review                             0
dtype: int64

In [15]:
mcd= mcd[mcd.policies_violated.notna()]

In [16]:
mcd.shape

(1471, 10)

In [17]:
mcd['rude'] = mcd.policies_violated.str.contains('RudeService').astype('int')

In [18]:
mcd.loc[0:4, ['policies_violated', 'rude']]

,policies_violated,rude
0,RudeService\rOrderProblem\rFilthy,1
1,RudeService,1
2,SlowService\rOrderProblem,0
3,na,0
4,RudeService,1


In [39]:
mcd.rude.value_counts()

0    968
1    503
Name: rude, dtype: int64

In [40]:
X = mcd.review
y = mcd.rude

In [41]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()


In [43]:
# fit and transform X_train into X_train_dtm
X_train_dtm = vect.fit_transform(X_train)
X_train_dtm.shape



(1103, 7301)

In [44]:
# transform X_test into X_test_dtm
X_test_dtm = vect.transform(X_test)
X_test_dtm.shape

(368, 7301)

In [24]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [25]:
# calculate the predicted probability of rude=1 for each testing set observation
y_pred_prob = nb.predict_proba(X_test_dtm)[:, 1]

In [26]:
# calculate the AUC
from sklearn import metrics
metrics.roc_auc_score(y_test, y_pred_prob)

0.8426005404546177

In [27]:
# repeat this task using a logistic regression model
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(solver='liblinear')
logreg.fit(X_train_dtm, y_train)
y_pred_prob = logreg.predict_proba(X_test_dtm)[:, 1]
metrics.roc_auc_score(y_test, y_pred_prob)

0.8233349229057383

In [29]:
def tokenize_test(vect):
        # create document-term matrices using the vectorizer
    X_train_dtm = vect.fit_transform(X_train)
    X_test_dtm = vect.transform(X_test)
        # print the number of features that were generated
    print('Features:', X_train_dtm.shape[1])
    # use Multinomial Naive Bayes to calculate predicted probabilities
    nb = MultinomialNB()
    nb.fit(X_train_dtm, y_train)
    y_pred_prob = nb.predict_proba(X_test_dtm)[:, 1]
    
    print('AUC:', metrics.roc_auc_score(y_test, y_pred_prob))


In [30]:
vect = CountVectorizer()
tokenize_test(vect)


Features: 7301
AUC: 0.8426005404546177


In [31]:
# tune CountVectorizer to increase the AUC
vect = CountVectorizer(stop_words='english', max_df=0.3, min_df=4)
tokenize_test(vect)


Features: 1732
AUC: 0.8621522810364012
